In [2]:
import omicsio
import pickle
import json
import shapely

In [3]:
with open("./input_data/all_visium_data/annotations/json_files/_SS12251_092534.json") as f:
    slide_polygons_json = json.load(f)

with open("./input_data/all_visium_data/preprocessed/cytassist_092534_40x.pkl", "rb") as f:
    slide = pickle.load(f)

In [4]:
def simple_svg_to_shapely_polygon(svg):
    """ Only works if there is only one <polygon/> in the svg"""
    quote_start = svg.find("\"") + 1
    quote_end = svg.rfind("\"")
    points_string = svg[quote_start:quote_end]
    points = []
    for xy_string in points_string.split(" "):
        x_string, y_string = xy_string.split(",")
        x = float(x_string)
        y = float(y_string)
        points.append((x, y))
    return shapely.Polygon(points)

samples = {key: {'intra': None, 'inter': None} for key in ['24', '35']}

sample_id = None
for annotation in slide_polygons_json:
    annotation_type = annotation['body'][0]['value']
    selector = annotation['target']['selector']['value']
    if annotation_type not in ['intra', 'inter']:
        sample_id = annotation_type
    else:
        samples[sample_id][annotation_type] = simple_svg_to_shapely_polygon(selector)

In [5]:
samples

{'24': {'intra': <POLYGON ((10447 9995.1, 11886 11004, 12869 11427, 13557.9 11992.5, 14942.3 ...>,
  'inter': <POLYGON ((10028.5 8623.8, 9112.7 10073.7, 8516 11529.4, 7313.7 16453.6, 744...>},
 '35': {'intra': <POLYGON ((49502.699 73283.398, 49220.602 70780.297, 48384.301 67406.898, 47...>,
  'inter': <POLYGON ((53854.301 72108.102, 53841 71951.898, 52586.301 67907.297, 52165....>}}

In [ ]:
# Highlighting cells from each polygon